#### Preparando entorno

In [1]:
import enum
print(enum.__file__)  
# standard library location should be something like 
# /usr/local/lib/python3.6/enum.py

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/enum.py


In [2]:
import sys
!{sys.executable} -m pip uninstall -y enum34

In [3]:
!{sys.executable} -m pip install awswrangler

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [4]:
!{sys.executable} -m pip install PyAthena

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [5]:
!{sys.executable} -m pip install geopandas

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


#### Configurar PyAthena

In [6]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
from datetime import datetime
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

In [7]:
import geopandas as gpd
import glob
import os

In [8]:
import awswrangler as wr
import numpy as np
import calendar
import boto3
s3 = boto3.resource('s3')

#### Seleccionar paises

In [9]:
paises = ['BO','EC','CL','VE','AR','CR','DO','SV','GT','HN','NI','PA','PY','PE','UY','BZ','GY','JM','TT','CO','MX','BR']
len(paises)

22

In [10]:
pais = 'BR'

#### Asignar poligonos a usuarios

In [11]:
# Nombre de categorias de pois
categories = ['all']
#categories = ['banks','all', 'education','finance','food','health']

# Asignando pois a tabla historica de usuarios
for cat in categories:
            
    table_poi = f'historico_hogares_pois_{cat}_{pais}'
    
    if cat == 'banks':
        
        query = f'''
                CREATE TABLE {table_poi} AS
                SELECT iso_country_code,
                        caid,
                        lat_final,
                        long_final,
                        id as id_poi
                FROM historico_hogares_{pais}
                CROSS JOIN pois_{cat}_{pais}
                WHERE ST_CONTAINS (ST_GeomFromLegacyBinary(pois_{cat}_{pais}.boundarypoi), ST_POINT(historico_hogares_{pais}.long_final, historico_hogares_{pais}.lat_final))
                ORDER BY  id'''
        cursor.execute(query)
        
        print(f'Generacion de tabla {table_poi}: terminado')
        
    else:
        
        query = f'''CREATE TABLE {table_poi} AS
                SELECT iso_country_code,
                        caid,
                        lat_final,
                        long_final,
                        idn as id_poi,
                        poi_type,
                        facility,
                        cd_geocmi
                FROM historico_hogares_{pais}
                CROSS JOIN pois_{cat}_{pais}
                WHERE ST_CONTAINS (ST_GeomFromLegacyBinary(pois_{cat}_{pais}.boundarypoi), ST_POINT(historico_hogares_{pais}.long_final, historico_hogares_{pais}.lat_final))
                ORDER BY  idn'''
        cursor.execute(query)
        
        print(f'Generacion de tabla {table_poi}: terminado')

Generacion de tabla historico_hogares_pois_all_BR: terminado


#### Eliminar tablas

pais = 'BR'
try:
    tabla_datos_pais = f"historico_hogares_pois_banks_{pais}"
    query = f"DROP TABLE {tabla_datos_pais}"
    cursor.execute(query)

    tabla_datos_pais = f"historico_hogares_pois_all_{pais}"
    query = f"DROP TABLE {tabla_datos_pais}"
    cursor.execute(query)

    tabla_datos_pais = f"historico_hogares_pois_education_{pais}"
    query = f"DROP TABLE {tabla_datos_pais}"
    cursor.execute(query)

    tabla_datos_pais = f"historico_hogares_pois_finance_{pais}"
    query = f"DROP TABLE {tabla_datos_pais}"
    cursor.execute(query)

    tabla_datos_pais = f"historico_hogares_pois_food_{pais}"
    query = f"DROP TABLE {tabla_datos_pais}"
    cursor.execute(query)

    tabla_datos_pais = f"historico_hogares_pois_health_{pais}"
    query = f"DROP TABLE {tabla_datos_pais}"
    cursor.execute(query)
        
except:

    print(f" Eliminado ")